# Selecting Rows with ORM

Pada saat menggunakan ORM untuk mengeksekusi statement, maka dapat dengan menggunakan <b>Session.execute()</b>.

In [1]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import declarative_base

engine = create_engine("sqlite+pysqlite:///example2.db", future=True)
Base = declarative_base(bind=engine)

In [2]:
class Users(Base):
    __table__ = Table("users", Base.metadata, autoload=engine)
    __tablename__ = "users"
    
    def __repr__(self):
        return f'User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})'
    
class Email(Base):
    __table__ = Table("email", Base.metadata, autoload=engine)
    __tablename__ = "email"
    
    def __repr__(self):
        return f'Email(id={self.id!r}, email_address={self.email_address!r})'

In [3]:
from sqlalchemy import select
from sqlalchemy.orm import Session

In [4]:
sel_stmt = select(Users)
print(sel_stmt)

SELECT users.id, users.name, users.fullname 
FROM users


In [5]:
with Session(engine) as session:
    for row in session.execute(sel_stmt):
        print(row)

(User(id=1, name='spongebob', fullname='spongebob squarepants'),)
(User(id=2, name='sandy', fullname='sandy cheeks'),)
(User(id=3, name='patrick', fullname='patrick star'),)


In [7]:
sel_stmt = select(Users).where(Users.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(sel_stmt):
        print(row)

(User(id=1, name='spongebob', fullname='spongebob squarepants'),)


# SELECT Individual Columns

In [8]:
sel_stmt = select(Users.name)
print(sel_stmt)

SELECT users.name 
FROM users


In [12]:
with Session(engine) as session:
    result = session.execute(sel_stmt)
    print(result.fetchall())

[('spongebob',), ('sandy',), ('patrick',)]


## SELECT from two table

SELECT statement di bawah ini melakukan SELECT kolom name pada tabel users dan mengkombinasikan dengan seluruh kolom yang ada pada tabel email.

In [24]:
engine_core = create_engine("sqlite+pysqlite:///example.db", future=True)
with Session(engine_core) as session:
    result = session.execute(
        select(Users.name, Email).
        where(Users.id == Email.user_id).
        order_by(Email.id)
    )
    
    for row in result.all():
        print(row)

('spongebob', Email(id=1, email_address='spongebob@aol.com'))
('sandy', Email(id=2, email_address='sandy@aol.com'))
('patrick', Email(id=3, email_address='patrick@aol.com'))


## Selecting from Labeled SQL Expressions

In [26]:
sel_stmt = select(
               ("Username: " + Users.name).label("username")
           ).order_by(Users.name)
    
with Session(engine) as session:
    for row in session.execute(sel_stmt):
        print(f'{row.username}')


Username: patrick
Username: sandy
Username: spongebob


## Selecting with Textual Column Expressions

In [30]:
from sqlalchemy import text

sel_stmt = (
    select(
        text("'some phrase'"), Users.name
    ).order_by(Users.name)
)

with Session(engine) as session:
    print(session.execute(sel_stmt).all())

[('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'spongebob')]
